# Follow-up filtering
This primarily exists such that it is unecessary to run the full extraction process again while I find additional items to filter during the cleaning process

In [1]:
%load_ext autoreload
%autoreload 3

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import json
from pathlib import Path
from dutchanalyzer.config import *
from dutchanalyzer.utilities.utils import *
from dutchanalyzer.utilities.json_utils import *
from dutchanalyzer.utilities.replacement_utils import *
from pathlib import Path
from dotenv import load_dotenv
import datetime
from pprint import pprint
from tqdm import tqdm

In [3]:
from dutchanalyzer.utilities.pandas_utils import *

In [3]:
today = datetime.date.today().__format__("%d-%m-%y")
current_save_folder = Path(INTERIM_DATA_DIR, 'cleaning', 'wikt', str(today))

In [4]:
folders = ['EEF', 'ENF', 'NNF', 'NEF']


for f in folders:
    Path.mkdir(Path(current_save_folder, f), parents=True, exist_ok=True)

In [5]:
NNR_file = Path(NNR_DIR, 'NNR.jsonl')
NER_file = Path(NER_DIR, 'NER.jsonl')
EER_file = Path(EER_DIR, 'EER.jsonl')
ENR_file = Path(ENR_DIR, 'ENR.jsonl')

## Follow Up Filtering Nov 11

In [ ]:
def filter_obj(obj):
    obj.pop('wikipedia', '')
    obj.pop('instances', '')
    obj.pop('pos_tile', '')
    obj.pop('proverbs', '')
    obj.pop('anagrams', '')
    banned_pos_types = ['num', 'symbol', 'phrase', 'character', 'punct', 'abbrev', 'proverb']
    if obj.get('pos', '') in banned_pos_types:
        obj.clear()

In [7]:
def sort_df_columns(df, start_cols=['word', 'pos', 'lang_code', 'standard_lang', 'senses'], end_cols=['translations', 'lang', 'wl_code'], groups=['forms', 'etymology', 'nyms', 'categories']):
    df_cols = df.columns
    start = [c for c in start_cols if c in df_cols]
    end = [c for c in end_cols if c in df_cols]
    protected_cols = start 
    
    forms_grouping = ['form_of','forms', 'alt_of', 'inflection_templates', 'derived']
    etymology_grouping = ['etymology_templates', 'etymology_text', 'etymology_tree', 'etymology_number']
    nyms_grouping = ['synonyms', 'antonyms', 'hypernyms','hyponyms', 'troponyms', 'holonyms', 'meronyms']
    categories_grouping = ['categories', 'instances', 'links', 'related', 'topics']

    if groups:
        for group in groups:
            if 'forms' == group:
                forms_grouping = [c for c in forms_grouping if c in df_cols]
                protected_cols += forms_grouping
            if 'etymology' == group:
                etymology_grouping = [c for c in etymology_grouping if c in df_cols]
                protected_cols += etymology_grouping

            if 'nyms' == group:
                nyms_grouping = [c for c in nyms_grouping if c in df_cols]
                protected_cols += nyms_grouping

            if 'categories' == group:
                categories_grouping = [c for c in categories_grouping if c in df_cols]
                protected_cols += categories_grouping

    protected_cols = [c for c in protected_cols if c in df_cols]
    unprotected_cols = [c for c in df_cols if c not in protected_cols and c not in end]

    unprotected_cols.sort()
    new_cols = protected_cols + unprotected_cols + end
    df = df.loc[:, new_cols]
    return df

In [8]:
def process_obj(in_file, entries_out_file, wl_code, definitions_out_file=None, batch_size=1000, break_point=-1):
    batch = []
    entries_batch = []
    error_lines = []
    entry_id = 0
    with open(in_file, 'r', encoding='utf-8') as f:
        with open(entries_out_file, 'w+', encoding='utf-8') as out:
            
            for i, line in tqdm(enumerate(f)):
                if break_point > 0:
                    if i > break_point:
                        display(entries_batch)
                        break
                if line:
                    try:
                        obj = json.loads(line)
                        filter_obj(obj)
                        obj = sort_standardize_entry(obj)
                        if obj:
                            obj['entry_id'] = f"{wl_code}_{entry_id}"
                            entry_id += 1
                            obj['wl_code'] = wl_code

                            entries_batch.append(obj)

                            if len(entries_batch) > batch_size:
                                for entry in entries_batch:
                                    json.dump(entry, out, ensure_ascii=False)
                                    out.write('\n')
                                entries_batch = []
                            if definitions_out_file:
                                word_entry = extract_words_senses(obj)
                                batch.append(word_entry)
                                if len(batch) > batch_size:
                                    with open(definitions_out_file, 'a+', encoding='utf-8') as def_out: 
                                        for entry in batch:
                                            json.dump(entry, def_out, ensure_ascii=False)
                                            def_out.write('\n')
                                        batch = []
                    except Exception as e:
                        error_lines.append((i, obj))
                        display(line)
                        print("Error on line: ", i, " Error: ", e)
                        break
            if entries_batch:
                for entry in entries_batch:
                    json.dump(entry, out, ensure_ascii=False)
                    out.write('\n')  
            if batch and definitions_out_file:
                with open(definitions_out_file, 'a+', encoding='utf-8') as def_out: 
                    for entry in batch:
                        json.dump(entry, def_out, ensure_ascii=False)
                        def_out.write('\n')
    return entries_batch, batch, error_lines

In [69]:
file = EEF_FILE
temp_out = Path(current_save_folder, 'en', 'EEF', 'EEF.jsonl')
defs_file = Path(current_save_folder, 'en', 'EEF', 'EEF_definitions.jsonl')
wl_code = 'EEF'
entries_batch, batch, error_lines = process_obj(file, temp_out, wl_code, defs_file, 1000)

1240994it [03:23, 6085.15it/s]


### Other Files Output

In [66]:
file = ENF_FILE
temp_out = Path(current_save_folder, 'en', 'ENF', 'ENF.jsonl')
defs_file = Path(current_save_folder, 'en', 'ENF', 'ENF_definitions.jsonl')
wl_code = 'ENF'
enf_entries_batch, enf_batch, error_lines = process_obj(file, temp_out, wl_code, defs_file, 1000)

128545it [00:27, 4647.86it/s]


In [125]:
file = NNF_FILE
temp_out = Path(current_save_folder, 'nl', 'NNF', 'NNF.jsonl')
defs_file = Path(current_save_folder, 'nl', 'NNF', 'NNF_definitions.jsonl')
wl_code = 'NNF'
nn_entries_batch, nn_batch, error_lines = process_obj(file, temp_out, wl_code, defs_file, 1000)

603262it [01:50, 5447.05it/s] 


In [67]:
file = NEF_FILE
temp_out = Path(current_save_folder, 'nl', 'NEF', 'NEF.jsonl')
defs_file = Path(current_save_folder, 'nl', 'NEF', 'NEF_definitions.jsonl')
wl_code = 'NEF'
ne_entries_batch, ne_batch, error_lines = process_obj(file, temp_out, wl_code, defs_file, 1000)

16832it [00:01, 10716.85it/s]


## 2nd Filtering

In [112]:
def process_obj2(in_file, entries_out_file, batch_size=1000, break_point=-1):
    batch = []
    entries_batch = []
    error_lines = []
    entry_id = 0
    with open(in_file, 'r', encoding='utf-8') as f:
        with open(entries_out_file, 'w+', encoding='utf-8') as out:
            
            for i, line in tqdm(enumerate(f)):
                if break_point > 0:
                    if i > break_point:
                        display(entries_batch)
                        break
                if line:
                    try:
                        obj = json.loads(line)
                        filter_obj(obj)
                        
                        if obj:
                            entries_batch.append(obj)
                            if len(entries_batch) > batch_size:
                                for entry in entries_batch:
                                    json.dump(entry, out, ensure_ascii=False)
                                    out.write('\n')
                                entries_batch = []
                    except Exception as e:
                        error_lines.append((i, obj))
                        display(line)
                        print("Error on line: ", i, " Error: ", e)
                        break
                    
            if entries_batch:
                for entry in entries_batch:
                    json.dump(entry, out, ensure_ascii=False)
                    out.write('\n')  
            
    return entries_batch

In [134]:
file = Path(current_save_folder, 'en', 'EEF', 'EEF.jsonl')
temp_out = Path(current_save_folder, 'en', 'EEF', 'EEF_temp.jsonl')
entries_batch = process_obj2(file, temp_out)

1235112it [02:04, 9954.47it/s] 


In [126]:
file = Path(current_save_folder, 'en', 'ENF', 'ENF.jsonl')
temp_out = Path(current_save_folder, 'en', 'ENF', 'ENF_temp.jsonl')
enf_entries_batch = process_obj2(file, temp_out)

128067it [00:20, 6292.70it/s]


In [127]:
file = Path(current_save_folder, 'nl', 'NNF', 'NNF.jsonl')
temp_out = Path(current_save_folder, 'nl', 'NNF', 'NNF_temp.jsonl')
nn_entries_batch = process_obj2(file, temp_out)

599938it [01:06, 8970.40it/s] 


In [128]:
file = Path(current_save_folder, 'nl', 'NEF', 'NEF.jsonl')
temp_out = Path(current_save_folder, 'nl', 'NEF', 'NEF_temp.jsonl')
ne_entries_batch = process_obj2(file, temp_out)

16367it [00:00, 18762.22it/s]


In [135]:
for k, val in folders.items():
    for v in val:
        file = Path(current_save_folder, k, v, f'{v}.jsonl').__str__()
        temp_out = Path(current_save_folder, k, v, f'{v}_temp.jsonl')
        if temp_out.exists():
            os.remove(file)
            Path.rename(temp_out, file)


In [ ]:
for k, val in folders.items():
    for v in val:
        file = Path(current_save_folder, k, v, f'{v}.jsonl').__str__()
        temp_out = Path(current_save_folder, k, v, f'{v}.jsonl')
        if temp_out.exists():
            os.remove(file)
            Path.rename(temp_out, file)

In [162]:
temp_file = Path(current_save_folder, 'nl', 'NNF', f'NNF.jsonl')
file = Path(NNF_FILE).__str__()
if temp_file.exists():
    os.remove(file)
    Path.rename(temp_file, file)

In [163]:
temp_file = Path(current_save_folder, 'nl', 'NEF', f'NEF.jsonl')
file = Path(NEF_FILE).__str__()
if temp_file.exists():
    os.remove(file)
    Path.rename(temp_file, file)

In [164]:
temp_file = Path(current_save_folder, 'en', 'EEF', f'EEF.jsonl')
file = Path(EEF_FILE).__str__()
if temp_file.exists():
    os.remove(file)
    Path.rename(temp_file, file)

In [165]:
temp_file = Path(current_save_folder, 'en', 'ENF', f'ENF.jsonl')
file = Path(ENF_FILE).__str__()
if temp_file.exists():
    os.remove(file)
    Path.rename(temp_file, file)

## Sort Large Files

In [ ]:
def saveletterlist(letter_list, save_folder, letter):
    letter_file = Path(save_folder, 'letter_files', f'{letter}.jsonl')
    with open(letter_file, 'a+', encoding='utf-8') as sf: 
        for obj in letter_list:
            json.dump(obj, sf, ensure_ascii=False)
            sf.write('\n')

In [ ]:
sorted_eef = alpha_sort_large_file(EEF_FILE, Path(current_save_folder, 'en', 'EEF'))

In [ ]:
overwrite_file(EEF_FILE, sorted_eef)

In [ ]:
sorted_enf = alpha_sort_large_file(ENF_FILE, Path(current_save_folder, 'en', 'ENF'))

In [ ]:
overwrite_file(ENF_FILE, sorted_enf)

In [ ]:
sorted_nnf = alpha_sort_large_file(NNF_FILE, Path(current_save_folder, 'nl', 'NNF'))

In [ ]:
overwrite_file(NNF_FILE, sorted_nnf)

In [ ]:
sorted_nef = alpha_sort_large_file(NEF_FILE, Path(current_save_folder, 'nl', 'NEF'))

In [ ]:
overwrite_file(NEF_FILE, sorted_nef)

## Filter Sense Level Categories

In [13]:
nef_path = Path(NEF_FILE.__str__())

In [18]:
temp_file = overwrite_file_with_filter_operations(Path(WIKT_CLEANING_DIR, '22-11-25', 'en', 'EEF', 'EEF.jsonl'), filter_ops=['filter_sense_categories'], batch_size=10000)

Counting Lines: 100%|██████████| 598k/598k [00:00<00:00, 700MB/s]


Lines in file: 101


100%|██████████| 101/101 [00:00<00:00, 12057.06it/s]


aborted temp file saved as:  data\interim\cleaning\wikt\22-11-25\en\EEF\EEF_temp.jsonl


In [21]:
temp_file =  overwrite_file_with_filter_operations(EEF_FILE, filter_ops=['filter_sense_categories'])

Counting Lines: 100%|██████████| 9.80M/9.80M [00:00<00:00, 1.39GB/s]


Lines in file: 10001


  0%|          | 8/10001 [00:00<00:07, 1277.34it/s]


JSONDecodeError: Expecting ',' delimiter: line 1 column 168 (char 167)

## Pandas Analysis

In [24]:
def summarize_df(df: pd.DataFrame, name: str = None, num_top_values=10) -> pd.DataFrame:
    """
    Summarize a DataFrame similar to df.info(), but with:
      - shape
      - non-null count
      - unique count
      - top 10 value counts (as string)
    """
    summary_data = []

    for i, col in enumerate(df.columns):
        non_null = df[col].notna().sum()
        try: 
            unique_vals = df[col].nunique(dropna=True)
        except:
            unique_vals = 'List'
        top_vals = df[col].value_counts(dropna=False).head(num_top_values)
        top_vals_str = ", ".join(f"{idx} : {val}" for idx, val in top_vals.items())

        summary_data.append({
            "df_name": name,
            'col_num': i,
            "column": col,
            "dtype": str(df[col].dtype),
            "non_null": non_null,
            "unique": unique_vals,
            "top_vals": top_vals_str,
            "sh_rows": df.shape[0],
            "sh_cols": df.shape[1],
        })
    summary_df = pd.DataFrame(summary_data)
  
    return summary_df

In [49]:
pd.set_option('display.max_colwidth', 3) 
summarize_df(df, 'EEF', 5)

df_name  col_num               column    dtype  non_null unique  \
0   EEF      0       word                 object   101       37      
1   EEF      1       pos                  object   101        8      
2   EEF      2       lang_code            object   101        1      
3   EEF      3       standard_lang        object   101        1      
4   EEF      4       senses               object   101       List    
5   EEF      5       forms                object   81        List    
6   EEF      6       derived              object   47        List    
7   EEF      7       etymology_templates  object   87        List    
8   EEF      8       etymology_text       object   92        56      
9   EEF      9       synonyms             object   18        List    
10  EEF     10       antonyms             object    3        List    
11  EEF     11       hypernyms            object   14        List    
12  EEF     12       hyponyms             object   15        List    
13  EEF     13       holonyms             object    4        List    
14  EEF     14       meronyms             object    9        List    
15  EEF     15       categories           object   101       List    
16  EEF     16       related              object   41        List    
17  EEF     17       coordinate_terms     object    3        List    
18  EEF     18       descendants          object    9        List    
19  EEF     19       etymology_number     float64  64         6      
20  EEF     20       head_templates       object   101       List    
21  EEF     21       hyphenations         object   26        List    
22  EEF     22       instances            object    1        List    
23  EEF     23       sounds               object   100       List    
24  EEF     24       translations         object   45        List    
25  EEF     25       lang                 object   101        1      
26  EEF     26       wl_code              object   101        1      

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [25]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 27 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   word                 101 non-null    object 
 1   pos                  101 non-null    object 
 2   lang_code            101 non-null    object 
 3   standard_lang        101 non-null    object 
 4   senses               101 non-null    object 
 5   forms                81 non-null     object 
 6   derived              47 non-null     object 
 7   etymology_templates  87 non-null     object 
 8   etymology_text       92 non-null     object 
 9   synonyms             18 non-null     object 
 10  antonyms             3 non-null      object 
 11  hypernyms            14 non-null     object 
 12  hyponyms             15 non-null     object 
 13  holonyms             4 non-null      object 
 14  meronyms             9 non-null      object 
 15  categories           101 non-null    obj

In [27]:
def drop_df_column_groups(df, groups=['forms', 'etymology', 'nyms', 'categories'], individual_cols=['wl_code', 'lang']):
    df_cols = df.columns
    ind_cols = [c for c in individual_cols if c in df_cols]
    cols_to_drop = ind_cols
    
    forms_grouping = ['form_of','forms', 'alt_of', 'inflection_templates', 'derived']
    etymology_grouping = ['etymology_templates', 'etymology_text', 'etymology_tree']
    nyms_grouping = ['synonyms', 'antonyms', 'hypernyms','hyponyms', 'troponyms', 'holonyms', 'meronyms']
    categories_grouping = ['categories', 'links', 'related', 'topics']

    if groups:
        for group in groups:
            if 'forms' == group:
                forms_grouping = [c for c in forms_grouping if c in df_cols]
                cols_to_drop += forms_grouping
            if 'etymology' == group:
                etymology_grouping = [c for c in etymology_grouping if c in df_cols]
                cols_to_drop += etymology_grouping

            if 'nyms' == group:
                nyms_grouping = [c for c in nyms_grouping if c in df_cols]
                cols_to_drop += nyms_grouping

            if 'categories' == group:
                categories_grouping = [c for c in categories_grouping if c in df_cols]
                cols_to_drop += categories_grouping

    return df.drop(columns=cols_to_drop)

In [28]:
cut_df = drop_df_column_groups(df)

In [30]:
cut_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   word              101 non-null    object 
 1   pos               101 non-null    object 
 2   lang_code         101 non-null    object 
 3   standard_lang     101 non-null    object 
 4   senses            101 non-null    object 
 5   coordinate_terms  3 non-null      object 
 6   descendants       9 non-null      object 
 7   etymology_number  64 non-null     float64
 8   head_templates    101 non-null    object 
 9   hyphenations      26 non-null     object 
 10  instances         1 non-null      object 
 11  sounds            100 non-null    object 
 12  translations      45 non-null     object 
dtypes: float64(1), object(12)
memory usage: 10.4+ KB


In [39]:
def return_non_na_drop_cols(df, col):
    new_df = df[~df[col].isna()]
    return new_df.dropna(axis=1, how='all')

In [ ]:
temp_df = return_non_na_drop_cols(cut_df, 'coordinate_terms')

In [41]:
temp_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Index: 3 entries, 0 to 51
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   word              3 non-null      object 
 1   pos               3 non-null      object 
 2   lang_code         3 non-null      object 
 3   standard_lang     3 non-null      object 
 4   senses            3 non-null      object 
 5   coordinate_terms  3 non-null      object 
 6   etymology_number  2 non-null      float64
 7   head_templates    3 non-null      object 
 8   hyphenations      1 non-null      object 
 9   sounds            3 non-null      object 
 10  translations      1 non-null      object 
dtypes: float64(1), object(10)
memory usage: 288.0+ bytes


In [44]:
pd.set_option('display.max_colwidth', 3) 
temp_df.loc[:, ['word', 'senses','coordinate_terms']]

,word,senses,coordinate_terms
0,dictionary,"[{'glosses': ['A reference work listing words or names from one or more languages, usually ordered alphabetically, explaining each word's meanings or senses, oftentimes also containing information on its etymology, pronunciation, usage, semantic relations, translations, as well as other relevant information.'], 'categories': ['English terms with quotations', 'English terms with usage examples'], 'coordinate_terms': [{'word': 'thesaurus'}], 'hypernyms': [{'word': 'wordbook'}], 'links': [['reference work', 'reference work'], ['word', 'word'], ['name', 'name'], ['language', 'language'], ['alphabetically', 'alphabetically'], ['meanings', 'meaning#Noun'], ['senses', 'sense#English:_any_particular_meaning_of_a_word'], ['etymology', 'etymology#Noun'], ['pronunciation', 'pronunciation#Noun'], ['usage', 'usage#English:_language'], ['translations', 'translation#Noun']]}, {'glosses': ['A reference work on a particular subject or activity in which the entries are arranged alphabetically; an alphabetical encyclopedia.'], 'categories': ['English terms with usage examples'], 'links': [['reference work', 'reference work'], ['subject', 'subject'], ['activity', 'activity'], ['alphabetical', 'alphabetical'], ['encyclopedia', 'encyclopedia']], 'raw_glosses': ['(by extension) A reference work on a particular subject or activity in which the entries are arranged alphabetically; an alphabetical encyclopedia.'], 'tags': ['broadly']}, {'glosses': ['A person or thing regarded as a repository or compendium of information.'], 'links': [['person', 'person'], ['thing', 'thing'], ['repository', 'repository'], ['compendium', 'compendium'], ['information', 'information']], 'raw_glosses': ['(figurative) A person or thing regarded as a repository or compendium of information.'], 'tags': ['figuratively']}, {'glosses': ['The collection of words used or understood by a particular person; vocabulary.'], 'categories': ['English derogatory terms'], 'links': [['of', 'of#English'], ['derogatory', 'derogatory'], ['collection', 'collection'], ['word', 'word'], ['person', 'person'], ['vocabulary', 'vocabulary']], 'qualifier': 'frequently figurative', 'raw_glosses': ['(with of or possessive, frequently figurative, especially derogatory) The collection of words used or understood by a particular person; vocabulary.'], 'tags': ['derogatory', 'especially', 'possessive', 'with-of']}, {'glosses': ['A synchronic dictionary of a standardised language held to only contain words that are properly part of the language.'], 'categories': ['English terms with quotations'], 'links': [['the', 'the#English'], ['synchronic', 'synchronic']], 'qualifier': 'preceded by the', 'raw_glosses': ['(preceded by the) A synchronic dictionary of a standardised language held to only contain words that are properly part of the language.']}, {'glosses': ['An associative array, a data structure where each value is referenced by a particular key, analogous to words and definitions in a dictionary (sense 1).'], 'categories': ['English terms with quotations', 'en:Computing'], 'hyponyms': [{'word': 'hash table'}], 'links': [['computing', 'computing#Noun'], ['associative array', 'associative array']], 'raw_glosses': ['(computing) An associative array, a data structure where each value is referenced by a particular key, analogous to words and definitions in a dictionary (sense 1).'], 'topics': ['computing', 'engineering', 'mathematics', 'natural-sciences', 'physical-sciences', 'sciences']}]","[{'word': 'anagram dictionary', 'source': 'Thesaurus:dictionary'}, {'word': 'BD', 'source': 'Thesaurus:dictionary'}, {'word': 'biographical dictionary', 'source': 'Thesaurus:dictionary'}, {'word': 'concordance', 'source': 'Thesaurus:dictionary'}, {'word': 'guide', 'source': 'Thesaurus:dictionary'}, {'word': 'legend', 'source': 'Thesaurus:dictionary'}, {'word': 'onomasticon', 'source': 'Thesaurus:dictionary'}, {'word': 'rhyme book', 'source': 'Thesaurus:dictionary'}, {'word': 'rhym